In [ ]:
!pip install tensorflow tensorflow-gnn

In [2]:
import tensorflow as tf
from tensorflow_gnn.graph import GraphTensor
import tensorflow_gnn as tfgnn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Preparing the data
num_users = ratings_df['userId'].max() + 1
num_movies = ratings_df['movieId'].max() + 1

# Create features as one-hot vectors for users and movies
features = np.eye(num_users + num_movies, dtype=np.float32)
user_features = features[:num_users]
movie_features = features[num_users:num_users+num_movies]

# Create the edges
user_ids = ratings_df['userId'].values
movie_ids = ratings_df['movieId'].values + num_users  # offset movie IDs
ratings = ratings_df['rating'].values.astype(np.float32)

# Train-test split
train_idx, test_idx = train_test_split(np.arange(len(ratings)), test_size=0.1, random_state=42)

# Create GraphTensor
graph_tensor = GraphTensor.from_pieces(
    node_sets={
        'user': tfgnn.NodeSet.from_fields(features=user_features, sizes=[num_users]),
        'movie': tfgnn.NodeSet.from_fields(features=movie_features, sizes=[num_movies])
    },
    edge_sets={
        'rated': tfgnn.EdgeSet.from_fields(
            sizes=[len(ratings)],
            adjacency=tfgnn.Adjacency.from_indices(
                ('user', user_ids[train_idx]),
                ('movie', movie_ids[train_idx])
            ),
            features={'rating': ratings[train_idx]}
        )
    }
)

# Define the GAT model using TF-GNN layers
class GATModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.gat1 = tfgnn.keras.layers.GATConv(8, num_heads=8, dropout_rate=0.6)
        self.gat2 = tfgnn.keras.layers.GATConv(1, num_heads=1, dropout_rate=0.6)

    def call(self, graph):
        x = self.gat1(graph, ('user', 'rated', 'movie'))
        x = tf.nn.elu(x)
        x = self.gat2(x)
        return x

model = GATModel()
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(graph_tensor, epochs=200)

# Evaluate the model
test_graph_tensor = GraphTensor.from_pieces(
    node_sets={
        'user': tfgnn.NodeSet.from_fields(features=user_features, sizes=[num_users]),
        'movie': tfgnn.NodeSet.from_fields(features=movie_features, sizes=[num_movies])
    },
    edge_sets={
        'rated': tfgnn.EdgeSet.from_fields(
            sizes=[len(ratings)],
            adjacency=tfgnn.Adjacency.from_indices(
                ('user', user_ids[test_idx]),
                ('movie', movie_ids[test_idx])
            ),
            features={'rating': ratings[test_idx]}
        )
    }
)

predictions = model.predict(test_graph_tensor)
actual = test_graph_tensor.edge_sets['rated']['rating']

# Metrics calculation
rmse = np.sqrt(mean_squared_error(actual, predictions))
mae = mean_absolute_error(actual, predictions)
print(f'RMSE: {rmse}, MAE: {mae}')


ModuleNotFoundError: No module named 'tensorflow_gnn'